In [ ]:
import os

folder_path = "Summary_Data/"
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files)

In [ ]:
import pandas as pd
dfs = []
for i in files:
    temp_df = pd.read_csv('Summary_Data/' + i)
    dfs.append(temp_df)
combined_df = pd.concat(dfs, ignore_index=True)

In [ ]:
combined_df.to_csv("Summary_n_Transcript_data.csv",index=False)

In [ ]:
import json

def clean_transcript(text):
    try:
        s_ind = text.index("{")
        e_ind = text.index("}")
    except ValueError:
        return {}

    #text = text[s_ind:e_ind+1]
    text = text.replace('\\n', '\n')
    text = text.replace('\\\'', '\'')
    text = text.replace('\n', '')
    text = text.replace('\\', '')
    text = text.replace("   ", '')

    try:
        dic = json.loads(text)
    except json.JSONDecodeError:
        return {}

    return dic


In [ ]:
def clean_summary(text):
    text = text.replace('\\n', '\n')
    text = text.replace('\\\'', '\'')
    text = text.replace('\n', '')
    text = text.replace('\\', '')
    text = text.replace("   ", '')

    try:
        dic = json.loads(text)
    except json.JSONDecodeError:
        return {}

    return dic


In [ ]:
combined_df['Cleaned_Transcript'] = combined_df['transcript'].map(clean_transcript)
combined_df['Cleaned_Summary'] = combined_df['summary'].map(clean_summary)

In [ ]:
combined_df = combined_df[combined_df['Cleaned_Summary'] != {}]

In [ ]:
combined_df = combined_df[['Cleaned_Transcript','Cleaned_Summary']]
combined_df.rename(columns={'Cleaned_Transcript':'Meeting Transcript','Cleaned_Summary':'Summary'},inplace=True)

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
""".strip()

def generate_training_prompt(
    Transcript: str, Summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{Transcript.strip()}

### Response:
{Summary}
""".strip()

In [ ]:
combined_df['text'] = combined_df.apply(lambda x: generate_training_prompt(x['Meeting Transcript'],x['Summary']),axis=1)

In [ ]:
combined_df.to_csv("MOM_Summary.csv",index=False)